In [52]:
import torch
import torch.nn as nn             #muntar xarxes (capes, activacions, backpropagació de gradients...)
import torch.optim as optim       #escollir optimitzador que recalcularà els pesos
import torch.nn.functional as F   #cridar directament a funcions sense acudir a les classes
#import torch.utils.data as data   #muntar dataloaders que generaran els batches de dades
from torch.utils.data import Dataset, DataLoader
import torchvision  #eines diverses per descarregar bases de dades, transformar dades...

import matplotlib.pyplot as plt   #mostrar, plotejar, displayar dades i imatges
from tqdm import tqdm
import glob
import numpy as np
import cv2

In [53]:
batch_size =    5                 #per simplificar farem servir mateixa mida per training i test 
learning_rate = 0.01                #tasa d'aprenentatge
momentum =      0.1                  #paràmetre pel cas de l'optimitzador SGD (Stochastic gradient descent)
n_epochs =      10                  #vegades que la xarxa veurà totes les dades d'entrenament
criterium = nn.CrossEntropyLoss() #loss function 

In [54]:
class CustomDataset(Dataset):
    def __init__(self, path):
        self.imgs_path = path
        file_list = glob.glob(self.imgs_path + "*")
        print(file_list)
        self.data = []
        for class_path in file_list:
            class_name = class_path[len(path):]
            for img_path in glob.glob(class_path + "/*.jpg"):
                self.data.append([img_path, class_name])
        #print(self.data)
        for d in self.data:
            print(d)
        self.class_map = {"negativo" : 0, "positivo": 1, "tijeras": 2}
        self.img_dim = (480, 480)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = cv2.imread(img_path)
        img = cv2.resize(img, self.img_dim)
        class_id = self.class_map[class_name]
        img_tensor = torch.from_numpy(img)
        img_tensor = img_tensor.permute(2, 0, 1)
        class_id = torch.tensor([class_id])
        return img_tensor, class_id
        #return img_tensor.float(), class_id.float()     # quizas necesario

In [55]:
dataset = CustomDataset("frames/")

['frames\\negativo', 'frames\\positivo', 'frames\\tijeras']
['frames\\negativo\\IMG_0000.jpg', 'negativo']
['frames\\negativo\\IMG_0005.jpg', 'negativo']
['frames\\negativo\\IMG_0010.jpg', 'negativo']
['frames\\negativo\\IMG_0015.jpg', 'negativo']
['frames\\negativo\\IMG_0020.jpg', 'negativo']
['frames\\negativo\\IMG_0025.jpg', 'negativo']
['frames\\negativo\\IMG_0030.jpg', 'negativo']
['frames\\negativo\\IMG_0035.jpg', 'negativo']
['frames\\negativo\\IMG_0040.jpg', 'negativo']
['frames\\negativo\\IMG_0045.jpg', 'negativo']
['frames\\negativo\\IMG_0050.jpg', 'negativo']
['frames\\negativo\\IMG_0055.jpg', 'negativo']
['frames\\negativo\\IMG_0060.jpg', 'negativo']
['frames\\negativo\\IMG_0065.jpg', 'negativo']
['frames\\negativo\\IMG_0070.jpg', 'negativo']
['frames\\negativo\\IMG_0075.jpg', 'negativo']
['frames\\negativo\\IMG_0080.jpg', 'negativo']
['frames\\positivo\\IMG_0000.jpg', 'positivo']
['frames\\positivo\\IMG_0005.jpg', 'positivo']
['frames\\positivo\\IMG_0010.jpg', 'positivo']


In [56]:
dataset[2]

(tensor([[[255, 255, 255,  ..., 253, 253, 253],
          [255, 255, 255,  ..., 253, 253, 253],
          [255, 255, 255,  ..., 253, 253, 253],
          ...,
          [254, 254, 254,  ..., 129, 139, 149],
          [254, 254, 254,  ..., 131, 141, 151],
          [254, 254, 254,  ..., 132, 144, 153]],
 
         [[255, 255, 255,  ..., 253, 253, 253],
          [255, 255, 255,  ..., 253, 253, 253],
          [255, 255, 255,  ..., 253, 253, 253],
          ...,
          [255, 255, 255,  ..., 123, 133, 143],
          [255, 255, 255,  ..., 125, 135, 145],
          [255, 255, 255,  ..., 125, 135, 144]],
 
         [[255, 255, 255,  ..., 253, 253, 253],
          [255, 255, 255,  ..., 253, 253, 253],
          [255, 255, 255,  ..., 253, 253, 253],
          ...,
          [253, 253, 253,  ..., 172, 182, 192],
          [253, 253, 253,  ..., 176, 184, 194],
          [253, 253, 253,  ..., 176, 185, 195]]], dtype=torch.uint8),
 tensor([0]))

In [57]:
data_loader = DataLoader(dataset, batch_size=4, shuffle=True) #no_workers, pin_memory, drop_last

In [58]:
for imgs, labels in data_loader:
    print("Batch of images has shape: ",imgs.shape)
    print("Batch of labels has shape: ", labels.shape)

Batch of images has shape:  torch.Size([4, 3, 480, 480])
Batch of labels has shape:  torch.Size([4, 1])
Batch of images has shape:  torch.Size([4, 3, 480, 480])
Batch of labels has shape:  torch.Size([4, 1])
Batch of images has shape:  torch.Size([4, 3, 480, 480])
Batch of labels has shape:  torch.Size([4, 1])
Batch of images has shape:  torch.Size([4, 3, 480, 480])
Batch of labels has shape:  torch.Size([4, 1])
Batch of images has shape:  torch.Size([4, 3, 480, 480])
Batch of labels has shape:  torch.Size([4, 1])
Batch of images has shape:  torch.Size([4, 3, 480, 480])
Batch of labels has shape:  torch.Size([4, 1])
Batch of images has shape:  torch.Size([4, 3, 480, 480])
Batch of labels has shape:  torch.Size([4, 1])
Batch of images has shape:  torch.Size([4, 3, 480, 480])
Batch of labels has shape:  torch.Size([4, 1])
Batch of images has shape:  torch.Size([4, 3, 480, 480])
Batch of labels has shape:  torch.Size([4, 1])
Batch of images has shape:  torch.Size([4, 3, 480, 480])
Batch o

In [ ]:
# dades 
trans = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])   

train_data = torchvision.datasets.MNIST('/files/', train=True,  download=True, transform=trans)
test_data  = torchvision.datasets.MNIST('/files/', train=False, download=True, transform=trans) 


train_loader = data.DataLoader(train_data,batch_size=batch_size, shuffle=True)
test_loader =  data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

# exemple de mostra
idx = 3                       #un qualsevol per veure com són
input_example,target_example = train_data.__getitem__(idx)
print(input_example.size())       #mida de cada mostra/imatge/digit
plt.imshow(input_example[0,:,:], cmap='gray')
plt.title(str(target_example))
plt.show()

# exemple de batch
dataiter = iter(train_loader)
batch_images_example, batch_labels_example = dataiter.next()

plt.imshow(torchvision.utils.make_grid(batch_images_example)[0,:,:], cmap='gray')
plt.title('batch example')
plt.show()
print(batch_labels_example)